In [14]:
import pandas as pd
import telebot
from telebot import types

In [15]:
TOKEN_BOT = "6911877132:AAGi1Ljjo3whs4dAQm12pqmDwUd1aJVCEHg"

In [ ]:
# для изучения функций 
# bot = telebot.TeleBot(TOKEN_BOT) # new token
# @bot.message_handler(commands=['start'])  #, 'help'
# def handle_start(message):
#     mess = f'Привет!, <b> {message.from_user.first_name} <u>{message.from_user.last_name}</u> </b>'
#     bot.send_message(message.chat.id, mess, parse_mode = 'html' )

# @bot.message_handler()  # All message or text
# def handle_start(message):
#     if message.text.upper().lower() == "Привет":
#         bot.send_message(message.chat.id, f"И тебе привет {message.from_user.first_name}", parse_mode = 'html' )
#     elif message.text == "id":
#         bot.send_message(message.chat.id, f"Твой ID {message.from_user.id}", parse_mode = 'html' )
#     elif message.text == 'img':
#         photo = open("image_fx_ (BI).jpg", 'rb')
#         bot.send_photo(message.chat.id, photo=photo)
#     else:
#         bot.send_message(message.chat.id, f"Извини - не понимаю", parse_mode = 'html' )


# @bot.message_handler(content_types = ['photo'])
# def get_user_photo(message):
#     bot.send_message(message.chat.id, "Вы прислали картинку, а я ожидаю текст")

# @bot.message_handler(commands=['website']) # кнопка с ссылкой на сайт
# def website(message): # Для кнопок
#     markup = types.InlineKeyboardMarkup()
#     markup.add(types.InlineKeyboardButton("Перейти на сайт", url="ya.ru"))
#     bot.send_message(message.chat.id, f"Твой ID {message.from_user.id} переходи", reply_markup=markup)

# bot.polling(none_stop = True, interval = 0)

Last TRY!!!

In [23]:
univercity_df = pd.DataFrame(columns= ['course', "group", "nm"], data=[[1,9402, 25], [1,9403, 20], [2,9302,15]])
univercity_df.loc[len(univercity_df)] = [2,9308,18]
univercity_df.loc[len(univercity_df)] = [3,9208,7]
univercity_df.loc[len(univercity_df)] = [2,9309,17]
display(univercity_df) #[univercity_df["course"] == 2][0:1].group.values[0]
univercity_df[univercity_df["course"] == 2]["group"].values[0:][0]
univercity_df[univercity_df["group"] == 9308]["nm"].values[0:][0]

,course,group,nm
0,1,9402,25
1,1,9403,20
2,2,9302,15
3,2,9308,18
4,3,9208,7
5,2,9309,17


18

In [ ]:
bot = telebot.TeleBot(TOKEN_BOT)
user_state = {}  # Словарь для хранения состояния каждого пользователя

@bot.message_handler(commands=['start'])
def start_command(message):
    user_id = message.from_user.id
    user_state[user_id] = {}  # инициализация состояния
    user_state[user_id]['name'] = message.from_user.first_name
    name = user_state[user_id]['name']
    show_main_menu(message.chat.id, name)

def show_main_menu(chat_id, name):
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    btn1 = types.InlineKeyboardButton("Курс_1", callback_data="course_1")
    btn2 = types.InlineKeyboardButton("Курс_2", callback_data="course_2")
    btn3 = types.InlineKeyboardButton("Курс_3", callback_data="course_3")
    btn4 = types.InlineKeyboardButton("Курс_4", callback_data="course_4")
    keyboard.add(btn1, btn2, btn3, btn4)
    bot.send_message(chat_id, f'''Привет! {name} выбери курс ''', reply_markup=keyboard)

def show_submenu_category1(chat_id, course_number):
    # получаем список групп для курса
    groups = univercity_df[univercity_df["course"] == int(course_number)]["group"].values # отбор курса добавить
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    # btn_back = types.InlineKeyboardButton("🔙 Назад", callback_data="back_main")
    for grp in groups[:]:  # берем первые две группы а надо все!!!!
        btn = types.InlineKeyboardButton(f"Группа_{grp}", callback_data=f"group_{grp}")
        keyboard.add(btn)
    bot.send_message(chat_id, "Выберите группу:", reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: call.data.startswith(('course_', 'group_')))
def callback_query(call):
    user_id = call.from_user.id
    data = call.data  # например, "course_1" или "group_101"
    
    # если выбираем курс
    if data.startswith("course_"):
        course_number = data.split("_")[1]
        user_state[user_id]['course'] = f"Курс_{course_number}"
        # отправляем подменю групп
        show_submenu_category1(call.message.chat.id, course_number)
        bot.answer_callback_query(call.id)  # убираем "часики" на кнопке

    # если выбираем группу
    elif data.startswith("group_"):
        group_number = data.split("_")[1]
        user_state[user_id]['group'] = f"Группа_{group_number}"
        state = user_state[user_id]
        # достаём численность группы из dataframe
        nm = univercity_df[univercity_df['group'] == int(group_number)]['nm'].values[0]
        text = (f"Ваш курс: {state['course']}\n"
                f"Ваша группа: {state['group']}\n"
                f"Численность группы: {nm}")
        bot.send_message(call.message.chat.id, text)
        bot.answer_callback_query(call.id)


bot.polling()